# ⚡️ QuickStart: Scene Index

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/nb/main/guides/video/scene-index/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
---

### 📦  Installing packages   

In [ ]:
!pip install videodb

### 🔑 API Keys

In [4]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

### 🌐 Connect to VideoDB

In [5]:
from videodb import connect

conn = connect()
coll = conn.get_collection()

### 🎥  Upload Video

In [6]:
video = coll.upload(url="https://www.youtube.com/watch?v=LejnTJL173Y")

## 📇 Index Video
---

In [ ]:
video.index_scenes()

## 🔍 Search 
---

In [16]:
search_queries = ["drinking", "nudity", "blood", "fight", "smoking", "guns"]
res = [video.search(query, search_type="scene") for query in search_queries]

## View Results
---

In [17]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset
from videodb import play_stream

# Initialize timeline with connection
timeline = Timeline(conn)

# Add video shots to the timeline
for search_result in res:
    for shot in search_result.shots:
        video_asset = VideoAsset(asset_id=shot.video_id, start=shot.start, end=shot.end)
        timeline.add_inline(video_asset)

# Generate the stream URL and play the stream
stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/d8a4ae01-18da-4d46-9cd5-9ac0607c879b.m3u8'